In [1]:
#Basic 
import pandas as pd
import regex as re
import time
import requests

#For webscrape
from bs4 import BeautifulSoup
#from selenium import *
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [2]:
#Base url to get all the storkreds and valgkreds
URL = "https://www.dr.dk/nyheder/politik/folketingsvalg/din-stemmeseddel"
r = requests.get(URL)
soup = BeautifulSoup(r.content, "html.parser")

In [3]:
#Get links to each valgkreds
linksToKredse = {} #Key = link, value = [storkreds, valgkreds]
for storkreds in soup.find_all(class_="AccordionGrid_container___2Vv3"):
    storkredsDiv = storkreds.find('div') 
    storkredsName = storkredsDiv.get('id')
    for valgkreds in storkredsDiv.find_all(class_="AccordionGrid_cellWrapper__sGlRr"):
        valgkredsName = valgkreds.find('a')['aria-label']
        link = valgkreds.find('a')['href']
        linksToKredse['https://www.dr.dk'+link] = [storkredsName[10:], valgkredsName[:-20]]

In [4]:
def valgkreds(dic):
  """
  Returns list dicts of all the links for each of the candidates
  in each of the valgkredse
  """
  listOfValgkredse = []

  for key, value in dic.items():
    valgkreds = {}
    URL = key
    r = requests.get(URL)
    soup = BeautifulSoup(r.content, "html.parser")
    time.sleep(2)
    for parti in soup.find_all(class_="AccordionGrid_container___2Vv3"): #All parti containers
        partiName = parti.find('div')['aria-labelledby'][17:]
        for candidate in parti.find_all(class_="AccordionGrid_cellWrapper__sGlRr"): #All candidate containers
            candidateName = candidate.find('a')['aria-label'][:-22]
            candidateLink = 'https://www.dr.dk'+ candidate.find('a')['href']
            valgkreds[candidateLink] = [value[0], value[1] ,partiName, candidateName]
    listOfValgkredse.append(valgkreds)
  return listOfValgkredse

In [5]:
l = valgkreds(linksToKredse)

In [6]:
#Create an empty dataframe with each valgkreds
df = pd.DataFrame()

for i in l:
    temp = pd.DataFrame.from_dict(i, orient='index', columns = ['Storkreds', 'Valgkreds', 'Parti', 'Kandidat'])
    df = pd.concat([df, temp])
df.to_csv('data/Valgkredse.csv')

In [7]:
#Get dict of all candidate

kandidates = {} #Key = link, value = storkreds, parti, navn

storkreds = df['Storkreds'].to_list()
parti = df['Parti'].to_list()
name = df['Kandidat'].to_list()
link = df.index.to_list()

for i in range(len(storkreds)):
    kandidates[link[i]] = [storkreds[i], parti[i], name[i]]

In [8]:
first = True
def find_answers_for_candidate(c_url, driver):
    """Given the link for a candidate return all of the candidate's answers"""
    global first
    # built in delay so that we aren't being evil.
    driver.get(c_url)
    if first == True:
        #click no to cookies
        noCook = driver.find_element(By.XPATH, "/html/body/div[1]/div/div/div/div/div/div/div/div/div/div/div[3]/div/div[1]/button")
        noCook.click()
        first = False
    # click show more button
    while True:
        try:
            # Wait for the button to be clickable
            wait = WebDriverWait(driver, 5)
            button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".CandidateAnswers_expandButton__FLs7A button.dre-button.dre-button--secondary")))
            # Click the button
            button.click()
        except:
            break
    source = driver.page_source
    soup = BeautifulSoup(source, 'lxml')
    q = soup.find_all(class_='Answer_candidateFirstName__XTbqP')
    return [re.search(r"svaret:([a-zA-Z ]*)", x["aria-label"]).group(1).strip() for x in q]


In [9]:
#Get all the answers of all the candidates

driver = webdriver.Chrome()
for key, value in kandidates.items():
    x = find_answers_for_candidate(key,driver)
    kandidates[key].extend(x)

In [ ]:
#Check how many candidates' answers are unavailable
unavailable = 0
for key, value in kandidates.items():
    if len(value) == 3:
        unavailable += 1
print(unavailable)

110


In [13]:
columns = ["Storkreds", "Party", "Name"]

for i in range(25):
    columns.append("Q"+str(i+1))

vaaData = pd.DataFrame.from_dict(kandidates, orient='index', columns=columns)
vaaData.to_csv("data/VAA.csv")